<a href="https://colab.research.google.com/github/aadityane93/Machine_Learning_Notebooks/blob/main/Sorting_line_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tsfresh getml --upgrade
from google.colab import files
import numpy as np
import pandas as pd
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table
from sklearn.model_selection import train_test_split, KFold, cross_val_predict, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score
import xgboost as xgb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.5/176.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 63.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0


In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
folder_id = '1BZJfMkmr5uyyL1iZGPQen-B3VTYrJjD0' #100 files
# folder_id = '1bxp8LJq3826UeOR44M9ZiDFX61Dp-xSf' #all files
# folder_id = '1Goeo7w4XQ_BRZVN8zRX3t11Ollg811VB' #10 files
folder_list = drive.ListFile({'q': "'{}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false".format(folder_id)}).GetList()


for folder in folder_list:
    folder_title = folder['title']
    print(f"Processing folder: {folder_title}")
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder['id'])}).GetList()

    for index, file in enumerate(file_list):
        file_title = file['title']
        new_file_name = f"{folder_title}-{index}"
        print(f"Downloading {file_title} as {new_file_name}")
        file.GetContentFile(new_file_name)

Processing folder: niO
Processing folder: biO
Processing folder: riO
Processing folder: wiO


In [4]:
index = 100

#test train percentage
test_train_split_percentage = 20    # __% percent of data files will be used to train the model

print ("total no. of files for each color  - ", index)
print ("total no. of files  - ", index*4)
print (" ")

#Index to seperarate the read data into train and test dataframes
index_train = (test_train_split_percentage/100)* index
index_train = int(index_train)
index_test = 100 - index_train

print (f"Percent of training data - {test_train_split_percentage}%")
print (f"No. of files for training - {index_train}")
print (" ")
print (f"Percent of testing data - {100 - test_train_split_percentage}%")
print (f"No. of files for testing - {index_test}")

total no. of files for each color  -  100
total no. of files  -  400
 
Percent of training data - 20%
No. of files for training - 20
 
Percent of testing data - 80%
No. of files for testing - 80


In [11]:
def read_file(filename):
    return pd.read_csv((filename), sep=',')
########################################################## INDEX ################################################

# Dictonaries to read the files
white_train = white_test = {}
blue_train = blue_test = {}
red_train = red_test = {}
nio_train = nio_test = {}

#Assigning df
white_train_df = white_test_df = pd.DataFrame()
blue_train_df = blue_test_df = pd.DataFrame()
red_train_df = red_test_df = pd.DataFrame()
nio_train_df = nio_test_df = pd.DataFrame()

# i=j=0
#Reading files for training
for j in range (index_train):
  white_train[j] = read_file(f"wiO-{j}")
  blue_train[j] = read_file(f"biO-{j}")
  red_train[j] = read_file(f"riO-{j}")
  nio_train[j] = read_file(f"niO-{j}")
  # print(j)

print("\n")
#Reading files for testing
for i in range (index_test):
  white_test[i] = read_file(f"wiO-{i+index_train}")
  blue_test[i] = read_file(f"biO-{i+index_train}")
  red_test[i] = read_file(f"riO-{i+index_train}")
  nio_test[i] = read_file(f"niO-{i+index_train}")
  # print (i+index_train)

In [12]:
# Color df for Train Dataset
df_color_train = pd.DataFrame()
for i in range(len(white_train)):
  df_color_train = pd.concat([df_color_train, pd.Series(0)])
for i in range(len(blue_train)):
  df_color_train = pd.concat([df_color_train, pd.Series(1)])
for i in range(len(red_train)):
  df_color_train = pd.concat([df_color_train, pd.Series(2)])
for i in range(len(nio_train)):
  df_color_train = pd.concat([df_color_train, pd.Series(3)])
df_color_train = df_color_train.astype(int)
df_color_train.columns = ['color']


# Color df for Test Dataset
df_color_test = pd.DataFrame()
for i in range(len(white_test)):
  df_color_test = pd.concat([df_color_test, pd.Series(0)])
for i in range(len(blue_test)):
  df_color_test = pd.concat([df_color_test, pd.Series(1)])
for i in range(len(red_test)):
  df_color_test = pd.concat([df_color_test, pd.Series(2)])
for i in range(len(nio_test)):
  df_color_test = pd.concat([df_color_test, pd.Series(3)])
df_color_test = df_color_test.astype(int)
df_color_test.columns = ['color']

In [13]:
print(df_color_train.shape)
print(df_color_test.shape)

(320, 1)
(320, 1)


In [14]:
# Merging df of different Colors into a single df for Train Dataset

q= 0
r= index_train
s = index_train * 2
t = index_train * 3


for i in range(index_train):
  print (' q - ',q,' r - ',r,' s - ',s,' t - ',t)
  white_train[i] = pd.DataFrame(white_train[i])
  white_train[i]['fileno'] = q
  white_train_df = pd.concat([white_train_df, white_train[i]])

  blue_train[i] = pd.DataFrame(blue_train[i])
  blue_train[i]['fileno'] = r
  blue_train_df = pd.concat([blue_train_df, blue_train[i]])

  red_train[i] = pd.DataFrame(red_train[i])
  red_train[i]['fileno'] = s
  red_train_df = pd.concat([red_train_df, red_train[i]])

  nio_train[i] = pd.DataFrame(nio_train[i])
  nio_train[i]['fileno'] = t
  nio_train_df = pd.concat([nio_train_df, nio_train[i]])

  q = q + 1
  r = r + 1
  s = s + 1
  t = t + 1
  # print (i)


# Merging df of different Colors into a single df for Test Dataset

q = t
r = q + index_test
s = q + index_test*2
t = t + index_test*3

print (" \n")

for j in range(index_test):
  print (' q - ',q,' r - ',r,' s - ',s,' t - ',t)
  white_test[j] = pd.DataFrame(white_test[j])
  white_test[j]['fileno'] = q
  white_test_df = pd.concat([white_test_df, white_test[j]])

  blue_test[j] = pd.DataFrame(blue_test[j])
  blue_test[j]['fileno'] = r
  blue_test_df = pd.concat([blue_test_df, blue_test[j]])

  red_test[j] = pd.DataFrame(red_test[j])
  red_test[j]['fileno'] = s
  red_test_df = pd.concat([red_test_df, red_test[j]])

  nio_test[j] = pd.DataFrame(nio_test[j])
  nio_test[j]['fileno'] = t
  nio_test_df = pd.concat([nio_test_df, nio_test[j]])

  q = q + 1
  r = r + 1
  s = s + 1
  t = t + 1
  # print (j)

 q -  0  r -  20  s -  40  t -  60
 q -  1  r -  21  s -  41  t -  61
 q -  2  r -  22  s -  42  t -  62
 q -  3  r -  23  s -  43  t -  63
 q -  4  r -  24  s -  44  t -  64
 q -  5  r -  25  s -  45  t -  65
 q -  6  r -  26  s -  46  t -  66
 q -  7  r -  27  s -  47  t -  67
 q -  8  r -  28  s -  48  t -  68
 q -  9  r -  29  s -  49  t -  69
 q -  10  r -  30  s -  50  t -  70
 q -  11  r -  31  s -  51  t -  71
 q -  12  r -  32  s -  52  t -  72
 q -  13  r -  33  s -  53  t -  73
 q -  14  r -  34  s -  54  t -  74
 q -  15  r -  35  s -  55  t -  75
 q -  16  r -  36  s -  56  t -  76
 q -  17  r -  37  s -  57  t -  77
 q -  18  r -  38  s -  58  t -  78
 q -  19  r -  39  s -  59  t -  79
 

 q -  80  r -  160  s -  240  t -  320
 q -  81  r -  161  s -  241  t -  321
 q -  82  r -  162  s -  242  t -  322
 q -  83  r -  163  s -  243  t -  323
 q -  84  r -  164  s -  244  t -  324
 q -  85  r -  165  s -  245  t -  325
 q -  86  r -  166  s -  246  t -  326
 q -  87  r - 

In [15]:
# for i in range (len(white_df)):
#   white_df['fileno'] = i
# for j in range (i, i + len(blue_df)):
#   blue_df['fileno'].append(j)
# for k in range (j,j+len(red_df)):
#   red_df['fileno'].append(j)
# for l in range (k, k+len(nio_df)):
#   nio_df['fileno'].append(j)

In [16]:
#Assigning color to the the dataframe
white_train_df ['color'] = 0
blue_train_df ['color'] = 1
red_train_df ['color'] = 2
nio_train_df ['color'] = 3

white_test_df ['color'] = 0
blue_test_df ['color'] = 1
red_test_df ['color'] = 2
nio_test_df ['color'] = 3

In [17]:
df_train = pd.concat([white_train_df, blue_train_df, red_train_df, nio_train_df])
df_test = pd.concat([white_test_df, blue_test_df, red_test_df, nio_test_df])


df_train = df_train.drop(columns=['Datum'])
df_test = df_test.drop(columns=['Datum'])

df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

#For tsfresh
x_train_df = df_train.drop(columns=['P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)','color'])
x_test_df = df_test.drop(columns=['P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)','color'])

# y = df['color']
# j=0
# y_aggregated = pd.DataFrame()
# for i in range(len(df['fileno'])):
#   if df['fileno'].values[i] ==



  # if df['color'].values[i] == j:
  #   y_aggregated = pd.concat([y_aggregated, pd.Series(j)])
  # elif df['color'].values[i] != j:
  #   j = j + 1
  #   y_aggregated = pd.concat([y_aggregated, pd.Series(j)])
  # else:
  #   print('error')


In [18]:
print(df_train['fileno'])

0         0
1         0
2         0
3         0
4         0
         ..
59834    79
59835    79
59836    79
59837    79
59838    79
Name: fileno, Length: 59839, dtype: int64


In [19]:
features_train =  extract_features(
    x_train_df,
    column_id="fileno",
    column_sort="Uhrzeit",
    impute_function=impute
)

Feature Extraction: 100%|██████████| 400/400 [05:31<00:00,  1.21it/s]


In [20]:
# feature_file = f'features-{index*4}.csv'
# features.to_csv(feature_file, sep=',')
# files.download(feature_file)

In [21]:
features_train.head()

,I_In__variance_larger_than_standard_deviation,I_In__has_duplicate_max,I_In__has_duplicate_min,I_In__has_duplicate,I_In__sum_values,I_In__abs_energy,I_In__mean_abs_change,I_In__mean_change,I_In__mean_second_derivative_central,I_In__median,...,Temp(°C)__fourier_entropy__bins_5,Temp(°C)__fourier_entropy__bins_10,Temp(°C)__fourier_entropy__bins_100,Temp(°C)__permutation_entropy__dimension_3__tau_1,Temp(°C)__permutation_entropy__dimension_4__tau_1,Temp(°C)__permutation_entropy__dimension_5__tau_1,Temp(°C)__permutation_entropy__dimension_6__tau_1,Temp(°C)__permutation_entropy__dimension_7__tau_1,Temp(°C)__query_similarity_count__query_None__threshold_0.0,Temp(°C)__mean_n_absolute_max__number_of_maxima_7
0,0.0,0.0,1.0,1.0,188.18,60.0938,0.042901,0.000340,0.000000,0.29,...,1.173481,1.838182,3.827333,1.225892,2.037041,2.814254,3.512233,4.105720,0.0,37.88
1,0.0,0.0,1.0,1.0,192.78,62.0622,0.041866,0.000303,0.000000,0.29,...,1.214053,1.871119,3.751347,1.090549,1.763087,2.407287,2.994598,3.542982,0.0,37.88
2,0.0,0.0,0.0,1.0,197.64,63.2942,0.044281,0.000232,0.000015,0.29,...,1.429405,2.089522,4.047602,1.020513,1.630517,2.205977,2.761791,3.299071,0.0,37.88
3,0.0,0.0,1.0,1.0,180.88,58.3126,0.040799,0.000425,0.000051,0.29,...,1.305995,1.931358,3.864304,1.105649,1.797467,2.466143,3.099256,3.653979,0.0,37.88
4,0.0,0.0,1.0,1.0,196.09,62.8797,0.043609,0.000328,0.000000,0.31,...,1.357481,1.946136,3.840977,1.044719,1.695590,2.304138,2.883785,3.377789,0.0,37.88


In [22]:
print(features_train.shape)
y_train_df= df_color_train['color']
y_train_df = y_train_df.reset_index(drop=True)
print(y_train_df.shape)

(80, 3915)
(320,)


In [23]:
y_train_df

,color
0,0
1,0
2,0
3,0
4,0
...,...
315,3
316,3
317,3
318,3


In [24]:
features_train = features_train.reset_index(drop=True)
features_train

,I_In__variance_larger_than_standard_deviation,I_In__has_duplicate_max,I_In__has_duplicate_min,I_In__has_duplicate,I_In__sum_values,I_In__abs_energy,I_In__mean_abs_change,I_In__mean_change,I_In__mean_second_derivative_central,I_In__median,...,Temp(°C)__fourier_entropy__bins_5,Temp(°C)__fourier_entropy__bins_10,Temp(°C)__fourier_entropy__bins_100,Temp(°C)__permutation_entropy__dimension_3__tau_1,Temp(°C)__permutation_entropy__dimension_4__tau_1,Temp(°C)__permutation_entropy__dimension_5__tau_1,Temp(°C)__permutation_entropy__dimension_6__tau_1,Temp(°C)__permutation_entropy__dimension_7__tau_1,Temp(°C)__query_similarity_count__query_None__threshold_0.0,Temp(°C)__mean_n_absolute_max__number_of_maxima_7
0,0.0,0.0,1.0,1.0,188.18,60.0938,0.042901,0.000340,0.000000,0.29,...,1.173481,1.838182,3.827333,1.225892,2.037041,2.814254,3.512233,4.105720,0.0,37.880000
1,0.0,0.0,1.0,1.0,192.78,62.0622,0.041866,0.000303,0.000000,0.29,...,1.214053,1.871119,3.751347,1.090549,1.763087,2.407287,2.994598,3.542982,0.0,37.880000
2,0.0,0.0,0.0,1.0,197.64,63.2942,0.044281,0.000232,0.000015,0.29,...,1.429405,2.089522,4.047602,1.020513,1.630517,2.205977,2.761791,3.299071,0.0,37.880000
3,0.0,0.0,1.0,1.0,180.88,58.3126,0.040799,0.000425,0.000051,0.29,...,1.305995,1.931358,3.864304,1.105649,1.797467,2.466143,3.099256,3.653979,0.0,37.880000
4,0.0,0.0,1.0,1.0,196.09,62.8797,0.043609,0.000328,0.000000,0.31,...,1.357481,1.946136,3.840977,1.044719,1.695590,2.304138,2.883785,3.377789,0.0,37.880000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.0,0.0,1.0,1.0,270.68,85.7514,0.046291,0.000023,-0.000034,0.31,...,1.421374,2.076108,3.938840,1.400296,2.347628,3.290005,4.170394,4.934246,0.0,38.380000
76,0.0,0.0,0.0,1.0,258.34,82.4962,0.046722,0.000024,-0.000024,0.31,...,1.204341,1.781229,3.708065,1.382771,2.330288,3.289047,4.135694,4.856656,0.0,38.380000
77,0.0,0.0,1.0,1.0,267.46,84.7066,0.045411,0.000023,0.000023,0.31,...,1.492522,2.117737,4.028743,1.441098,2.381666,3.335599,4.209756,4.971119,0.0,38.272857
78,0.0,0.0,1.0,1.0,260.88,83.2914,0.047453,0.000000,-0.000035,0.31,...,1.257899,1.905192,3.837543,1.297325,2.149362,3.001222,3.752400,4.384163,0.0,38.380000


In [25]:
selected_features = select_features(features_train, y_train_df)

AssertionError: X and y must contain the same number of samples.

In [ ]:
selected_features.head()

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(selected_features,y_train_df, test_size=0.5, random_state=42, stratify=y_train_df)

In [ ]:
model = RandomForestClassifier()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
def xgb_(X, y, num_folds=5):

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=24)

    f1_scores = []
    accuracies = []
    precisions = []

    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X)):
        print(f"Fold {fold_idx + 1}:")

        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Initialize and train the XGBoost model with regularization
        xgb1 = xgb.XGBClassifier(
            objective="multi:softmax",
            random_state=42,
            alpha=0.1,
            reg_lambda=0.1,
            max_depth=4,
            min_child_weight=10,
            subsample=0.8,
            colsample_bytree=0.8
        )

        # Fit the model
        xgb1.fit(X_train.to_numpy(), y_train.to_numpy().ravel())

        # Make predictions
        y_pred = xgb1.predict(X_test.to_numpy())

        # Ensure y_test is 1D for comparison
        y_test = y_test.to_numpy().ravel().astype(int)
        y_pred = y_pred.astype(int)

        # Calculate and store F1-score for this fold
        f1 = f1_score(y_test, y_pred, average='weighted')
        f1_scores.append(f1)

        # Calculate and store accuracy and precision for this fold
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        precision = precision_score(y_test, y_pred, average='weighted')
        precisions.append(precision)

        print(classification_report(y_test, y_pred))
        print(f"Fold {fold_idx + 1} F1-score: {f1:.4f}")
        print(f"Fold {fold_idx + 1} Accuracy: {accuracy:.4f}")
        print(f"Fold {fold_idx + 1} Precision: {precision:.4f}")

        # Calculate and print confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        print("Confusion Matrix:")
        print(cm)

        print("-" * 30)

    # Calculate and print average metrics across all folds
    avg_f1 = np.mean(f1_scores)
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)

    print(f"Average F1-score across {num_folds} folds: {avg_f1:.4f}")
    print(f"Average Accuracy across {num_folds} folds: {avg_accuracy:.4f}")
    print(f"Average Precision across {num_folds} folds: {avg_precision:.4f}")

    # Final model: Train on the entire dataset
    print("Training final model on the entire dataset...")
    X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(X, y, test_size=0.2, random_state=42)

    # Final model with regularization
    final_model = xgb.XGBClassifier(
        objective="multi:softmax",
        random_state=42,
        alpha=0.1,
        reg_lambda=0.1,
        max_depth=5,
        min_child_weight=10,
        subsample=0.8,
        colsample_bytree=0.8
    )

    # Train on the entire dataset
    final_model.fit(X_train_final.to_numpy(), y_train_final.to_numpy().ravel())

    # Evaluate the final model on the test set
    y_pred_final = final_model.predict(X_test_final.to_numpy())

    # Calculate accuracy, precision, confusion matrix for final model
    final_accuracy = accuracy_score(y_test_final, y_pred_final)
    final_precision = precision_score(y_test_final, y_pred_final, average='weighted')
    final_cm = confusion_matrix(y_test_final, y_pred_final)

    print(f"Final Model Accuracy: {final_accuracy:.4f}")
    print(f"Final Model Precision: {final_precision:.4f}")
    print("Final Model Confusion Matrix:")
    print(final_cm)

    # Save the final model
    # with open('XGB_final_model.pkl', 'wb') as f:
    #     pickle.dump(final_model, f)
    # files.download('XGB_final_model.pkl')



In [ ]:
xgb_ (selected_features, y_train_df)

In [ ]:
features_test = extract_features(
    x_test_df,
    column_id="fileno",
    column_sort="Uhrzeit",
    impute_function=impute
)

In [ ]:
#Using the same columns from the training data
features_test = features_test[selected_features.columns]

In [ ]:
print(features_test.shape)
y_test_df= df_color_test['color']
y_test_df = y_test_df.reset_index(drop=True)
print(y_test_df.shape)

In [ ]:
# model = RandomForestClassifier()
# model.fit(features_test, y_test)
y_pred = model.predict(features_test)
accuracy = accuracy_score(y_test_df, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test_df, y_pred))
print(confusion_matrix(y_test_df, y_pred))

In [ ]:
# xgb_ (features_test, y_test_df)

In [ ]:
# Read file into DataFrame

wio15_df = pd.DataFrame(pd.read_csv(('wiO-15'), sep=',', header=0))



wio15_df['fileno'] = 15
wio15_df = wio15_df.drop(columns=['Datum','P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)'])

extracted_features = extract_features(
    wio15_df,
    column_id="fileno",
    column_sort="Uhrzeit",
    impute_function=impute
)
wio_features = extracted_features[selected_features.columns]
y_pred = model.predict(wio_features)
print(y_pred)
accuracy = accuracy_score(pd.Series([0]), y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(pd.Series([0]), y_pred))
print(confusion_matrix(pd.Series([0]), y_pred))


In [ ]:
# #For getML
# x_train_df2 = df_train.drop(columns=['P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)'])
# x_test_df2 = df_test.drop(columns=['P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)'])

# # x_train_df2 = x_train_df.copy(deep=True)

# import getml

# getml.engine.launch()
# getml.set_project("sorting-line")

# # Split at the last colon
# x_train_df2["Uhrzeit"] = (
#     x_train_df2["Uhrzeit"]
#     .str.rsplit(pat=':', n=1)
#     .str[0]
# )

# # Convert to datetime with flexible parsing
# x_train_df2["Uhrzeit"] = pd.to_datetime(
#     x_train_df2["Uhrzeit"],
#     format='mixed',
#     errors='coerce'
# )
# pop = getml.data.DataFrame.from_pandas(x_train_df2, name = "pop")


# pop.set_role("fileno", getml.data.roles.join_key)
# pop.set_role(["I_In", "V_In", "I_Out(A)", "V_Out(V)", "Temp(°C)"], getml.data.roles.numerical)
# pop.set_role("Uhrzeit", getml.data.roles.time_stamp)
# pop.set_role("color", getml.data.roles.target)

In [ ]:
# x_train_df2

In [ ]:
# pop